# Selecionado apenas as linhas com o valor Strand positivo (+)

Aqui nos queremos selecionar de cada arquivo .gff3 as linhas em que o valor da coluna Strand seja '+'. Selecionadas essas linhas, queremos recuperar os cromossomos Chr (seqid = indica a qual cromossomo o elemento transponível pertence) e os valores da coluna Start (posição inicial do elemento dentro desse cromossomo) e End (posição final). No fim, queremos apenas as linhas com Strand positivo e seus valores em 'seqid', 'start' e 'end'.

In [36]:
import pandas as pd

# 1. Defina o caminho relativo para o arquivo
# (.. para subir da pasta 'notebooks' e /data/ para descer em 'data')
file_path = '../data/arquivos_TEAnnotation_brutos/TEAnnotationFinal_TIR.gff3'

# 2. Definir os nomes das colunas padrão do formato GFF3
column_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

# 3. Ler o arquivo
# sep='\t' porque arquivos GFF3 são separados por Tab
# comment='#' para ignorar linhas de cabeçalho/metadados
# header=None porque o arquivo não tem uma linha de cabeçalho com nomes
df = pd.read_csv(
    file_path,
    sep='\t',
    comment='#',
    header=None,
    names=column_names
)

# 4. Mostrar as primeiras linhas para verificar
print(df.head())

  seqid  source                               type      start        end  \
0     3  APTEdb  Class II subclass 1/TIR/MuLE-MuDR  189779027  189779256   
1     8  APTEdb  Class II subclass 1/TIR/MuLE-MuDR   67477834   67477956   
2     1  APTEdb  Class II subclass 1/TIR/MuLE-MuDR   73838768   73839001   
3     4  APTEdb  Class II subclass 1/TIR/MuLE-MuDR  201149345  201149414   
4     4  APTEdb  Class II subclass 1/TIR/MuLE-MuDR    3250281    3250511   

  score strand phase                                         attributes  
0     .      -     .  TE-Score=0.285;Software=RepeatModeler;Length=2...  
1     .      +     .  TE-Score=0.142;Software=RepeatModeler;Length=1...  
2     .      +     .  TE-Score=0.285;Software=RepeatModeler;Length=2...  
3     .      +     .  TE-Score=0.142;Software=RepeatModeler;Length=6...  
4     .      +     .  TE-Score=0.285;Software=RepeatModeler;Length=2...  


# Imports e Definições de Caminhos

In [37]:
import glob
import os
from Bio import Entrez
from Bio import SeqIO

# --- Configuração ---

# 1. Caminhos dos seus arquivos filtrados (onde você salvou os .tsv)
input_dir = '../data/arquivos_TEAnnotation_filtrados/'

# 2. Pasta para onde baixaremos os genomas FASTA
fasta_dir = '../data/fasta_genomas/'
os.makedirs(fasta_dir, exist_ok=True)

# 3. Nome do arquivo de saída final
output_dataset = '../data/TE_dataset_final.csv'

# 4. Email para o NCBI (OBRIGATÓRIO para usar a API)
# Substitua pelo seu e-mail.
Entrez.email = "luizgcorreiadosantos@usp.br"

# Consolidar Coordenadas (Pandas)

In [38]:
print("Iniciando Parte 1: Consolidando arquivos TSV...")

all_data = []

# Encontrar todos os seus arquivos .tsv filtrados
tsv_files = glob.glob(os.path.join(input_dir, '*.tsv'))

if not tsv_files:
    print(f"ERRO: Nenhum arquivo .tsv encontrado em '{input_dir}'")
    print("Por favor, execute o script de filtragem primeiro.")
else:
    for file_path in tsv_files:
        # 1. Ler o arquivo de coordenadas
        df = pd.read_csv(file_path, sep='\t')
        
        # 2. Extrair o nome da Classe do nome do arquivo
        # Ex: 'TEAnnotationFinal_TIR_filtrado.tsv' -> 'TIR'
        filename = os.path.basename(file_path)
        classe = filename.replace('TEAnnotationFinal_', '').replace('_filtrado.tsv', '')
        
        # 3. Adicionar a coluna "Classe"
        df['Classe'] = classe
        
        all_data.append(df)

    # 4. Combinar tudo em um DataFrame
    coords_df = pd.concat(all_data, ignore_index=True)

    print(f"Total de {len(coords_df)} coordenadas de TEs carregadas.")
    print("Classes encontradas:")
    print(coords_df['Classe'].value_counts())
    print(coords_df.head())

Iniciando Parte 1: Consolidando arquivos TSV...
Total de 341712 coordenadas de TEs carregadas.
Classes encontradas:
Classe
LTR         191815
TIR          83849
MITE         26350
Helitron     24769
LINE         11174
SINE          3755
Name: count, dtype: int64
  seqid      start        end    Classe
0     7   82856122   82857584  Helitron
1     1  239302471  239302834  Helitron
2     8    2261318    2261604  Helitron
3     5  124665404  124665608  Helitron
4     9   77998293   78002783  Helitron


# Baixar Genomas (BioPython.Entrez)

In [39]:
print("\nIniciando Parte 2: Baixando genomas do NCBI...")

# Mapeamento do Cromossomo ('seqid') para o ID de Acesso do GenBank
# Baseado na tabela do PDF  (corrigindo pequenos erros de OCR)
chr_accessions = {
    '1': 'LR618874.1',
    '2': 'LR618875.1',
    '3': 'LR618876.1',
    '4': 'LR618877.1',
    '5': 'LR618878.1',
    '6': 'LR618879.1',
    '7': 'LR618880.1',
    '8': 'LR618881.1',
    '9': 'LR618882.1',
    '10': 'LR618883.1',
    'MT': 'AY506529.1',
    'Pltd': 'X86563.2'
}

for chr_name, acc_id in chr_accessions.items():
    fasta_path = os.path.join(fasta_dir, f'chr_{chr_name}.fasta')
    
    # 1. Verifica se o arquivo JÁ FOI BAIXADO para não baixar de novo
    if not os.path.exists(fasta_path):
        print(f"Baixando: Cromossomo {chr_name} (ID: {acc_id})...")
        try:
            # 2. Busca (fetch) o registro no banco "nucleotide"
            handle = Entrez.efetch(db="nucleotide", id=acc_id, rettype="fasta", retmode="text")
            fasta_content = handle.read()
            handle.close()
            
            # 3. Salva o conteúdo em um arquivo .fasta
            with open(fasta_path, "w") as f:
                f.write(fasta_content)
            print(f"-> Salvo em: {fasta_path}")
            
        except Exception as e:
            print(f"ERRO ao baixar {acc_id}: {e}")
    else:
        print(f"Já existe: Cromossomo {chr_name} (Arquivo: {fasta_path})")

print("Download dos genomas concluído.")


Iniciando Parte 2: Baixando genomas do NCBI...
Já existe: Cromossomo 1 (Arquivo: ../data/fasta_genomas/chr_1.fasta)
Já existe: Cromossomo 2 (Arquivo: ../data/fasta_genomas/chr_2.fasta)
Já existe: Cromossomo 3 (Arquivo: ../data/fasta_genomas/chr_3.fasta)
Já existe: Cromossomo 4 (Arquivo: ../data/fasta_genomas/chr_4.fasta)
Já existe: Cromossomo 5 (Arquivo: ../data/fasta_genomas/chr_5.fasta)
Já existe: Cromossomo 6 (Arquivo: ../data/fasta_genomas/chr_6.fasta)
Já existe: Cromossomo 7 (Arquivo: ../data/fasta_genomas/chr_7.fasta)
Já existe: Cromossomo 8 (Arquivo: ../data/fasta_genomas/chr_8.fasta)
Já existe: Cromossomo 9 (Arquivo: ../data/fasta_genomas/chr_9.fasta)
Já existe: Cromossomo 10 (Arquivo: ../data/fasta_genomas/chr_10.fasta)
Já existe: Cromossomo MT (Arquivo: ../data/fasta_genomas/chr_MT.fasta)
Baixando: Cromossomo Pltd (ID: X86563.2)...
-> Salvo em: ../data/fasta_genomas/chr_Pltd.fasta
Download dos genomas concluído.


# Parte 3 - Carregar Genomas (BioPython.SeqIO)

In [40]:
print("\nIniciando Parte 3: Carregando genomas (FASTA) para a memória...")

genomes = {}

for chr_name in chr_accessions.keys():
    fasta_path = os.path.join(fasta_dir, f'chr_{chr_name}.fasta')
    
    try:
        # Usamos SeqIO.read() pois cada arquivo tem APENAS UMA sequência
        record = SeqIO.read(fasta_path, "fasta")
        
        # Armazenamos o objeto Seq (a sequência) no dicionário
        # A chave é o nome do cromossomo (ex: '1', 'MT')
        genomes[chr_name] = record.seq
        
        print(f"Carregado: chr {chr_name} ({len(record.seq)} bp)")
    except Exception as e:
        print(f"ERRO ao ler o arquivo {fasta_path}: {e}")

print("Genomas carregados.")


Iniciando Parte 3: Carregando genomas (FASTA) para a memória...
Carregado: chr 1 (308452471 bp)
Carregado: chr 2 (243675191 bp)
Carregado: chr 3 (238017767 bp)
Carregado: chr 4 (250330460 bp)
Carregado: chr 5 (226353449 bp)
Carregado: chr 6 (181357234 bp)
Carregado: chr 7 (185808916 bp)
Carregado: chr 8 (182411202 bp)
Carregado: chr 9 (163004744 bp)
Carregado: chr 10 (152435371 bp)
Carregado: chr MT (569630 bp)
Carregado: chr Pltd (140384 bp)
Genomas carregados.


# Extrair Sequências (Pandas + Bio.Seq)

In [41]:
print("\nIniciando Parte 4: Extraindo sequências de TE...")

def extrair_sequencia(linha):
    """
    Pega uma linha do DataFrame e extrai a sequência de DNA
    do dicionário de genomas.
    """
    try:
        # 1. Pega os dados da linha
        chr_name = str(linha['seqid'])
        start = linha['start']
        end = linha['end']
        
        # 2. Busca a sequência completa do cromossomo
        cromossomo_inteiro = genomes[chr_name]
        
        # 3. Fatiar (slice) a sequência
        # ATENÇÃO: Coordenadas GFF/FASTA são 1-based (começam em 1).
        # Slices de Python são 0-based (começam em 0).
        # Por isso, usamos [start-1:end].
        te_seq = cromossomo_inteiro[start-1:end]
        
        # 4. Retorna a sequência como texto
        return str(te_seq)
        
    except KeyError:
        # Caso o seqid (ex: 'Pt') não esteja no nosso mapa de downloads
        return f"ERRO: Cromossomo '{chr_name}' não encontrado nos arquivos FASTA."
    except Exception as e:
        return f"ERRO: {e}"

# Aplica a função para cada linha do DataFrame.
# Isso pode demorar alguns minutos se o DataFrame for grande!
coords_df['Sequência de TE'] = coords_df.apply(extrair_sequencia, axis=1)

print("Extração de sequências concluída.")
print(coords_df.head())


Iniciando Parte 4: Extraindo sequências de TE...
Extração de sequências concluída.
  seqid      start        end    Classe  \
0     7   82856122   82857584  Helitron   
1     1  239302471  239302834  Helitron   
2     8    2261318    2261604  Helitron   
3     5  124665404  124665608  Helitron   
4     9   77998293   78002783  Helitron   

                                     Sequência de TE  
0  GAGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAA...  
1  GTCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAA...  
2  ACGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATG...  
3  TATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTC...  
4  GTTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCAC...  


# Salvar Dataset Final

In [42]:
print("\nIniciando Parte 5: Gerando arquivo final...")

# 1. Renomear a coluna 'seqid' para 'Cromossomo'
final_df = coords_df.rename(columns={'seqid': 'Cromossomo'})

# 2. Selecionar apenas as colunas FINAIS pedidas no PDF 
colunas_finais = ['Cromossomo', 'Sequência de TE', 'Classe']
final_df = final_df[colunas_finais]

# 3. Salvar o arquivo final
final_df.to_csv(output_dataset, index=False)

print("---")
print(f"TRABALHO CONCLUÍDO! (Etapa de montagem de dados)")
print(f"Dataset final salvo em: {output_dataset}")
print(final_df.head())


Iniciando Parte 5: Gerando arquivo final...
---
TRABALHO CONCLUÍDO! (Etapa de montagem de dados)
Dataset final salvo em: ../data/TE_dataset_final.csv
  Cromossomo                                    Sequência de TE    Classe
0          7  GAGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAA...  Helitron
1          1  GTCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAA...  Helitron
2          8  ACGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATG...  Helitron
3          5  TATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTC...  Helitron
4          9  GTTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCAC...  Helitron


# Verificação de Erros

In [43]:
print("--- Verificação de Erros ---")

# 1. Filtra o DataFrame para encontrar linhas onde 'Sequência de TE' começa com 'ERRO'
erros_df = final_df[final_df['Sequência de TE'].str.startswith('ERRO')]

# 2. Conta o número total de erros
num_erros = len(erros_df)

if num_erros == 0:
    print("✅ Nenhuma linha com erro encontrada. Tudo certo!")
else:
    print(f"🚨 Atenção! Foram encontradas {num_erros} linhas com erro.")
    print("---")
    
    # 3. Mostra um resumo dos tipos de erro e suas contagens
    # Isso agrupa mensagens de erro idênticas (ex: 'Cromossomo X não encontrado')
    print("Resumo dos tipos de erro:")
    resumo_erros = erros_df['Sequência de TE'].value_counts()
    print(resumo_erros)
    print("---")
    
    # 4. Se houver poucos erros (ex: menos de 50), imprime o DataFrame de erros completo
    if num_erros < 50:
        print("DataFrame completo dos erros:")
        print(erros_df)
    else:
        print(f"Para ver todas as {num_erros} linhas de erro, rode o seguinte comando em uma nova célula:")
        print("print(erros_df.to_string())")

--- Verificação de Erros ---
🚨 Atenção! Foram encontradas 3378 linhas com erro.
---
Resumo dos tipos de erro:
Sequência de TE
ERRO: Cromossomo 'B73V4_ctg150' não encontrado nos arquivos FASTA.    228
ERRO: Cromossomo 'B73V4_ctg182' não encontrado nos arquivos FASTA.    177
ERRO: Cromossomo 'B73V4_ctg10' não encontrado nos arquivos FASTA.     138
ERRO: Cromossomo 'B73V4_ctg31' não encontrado nos arquivos FASTA.     111
ERRO: Cromossomo 'B73V4_ctg52' não encontrado nos arquivos FASTA.     106
                                                                     ... 
ERRO: Cromossomo 'B73V4_ctg47' não encontrado nos arquivos FASTA.       1
ERRO: Cromossomo 'B73V4_ctg246' não encontrado nos arquivos FASTA.      1
ERRO: Cromossomo 'B73V4_ctg228' não encontrado nos arquivos FASTA.      1
ERRO: Cromossomo 'B73V4_ctg172' não encontrado nos arquivos FASTA.      1
ERRO: Cromossomo 'B73V4_ctg160' não encontrado nos arquivos FASTA.      1
Name: count, Length: 241, dtype: int64
---
Para ver todas as